### Step 1 - Start Spark Session and Include additional configurations and common functions

In [ ]:
%run "../includes/configurations"

In [ ]:
%run "../includes/common_functions"

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize a Spark session
spark = SparkSession.builder.appName("LapTimes").getOrCreate()

### Step 2 - Read the foler with lap times files using the Spark Dataframe Reader

In [9]:
lap_times_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("driverId", IntegerType(), False),
    StructField("position", IntegerType(), False),
    StructField("lap", IntegerType(), False),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True)
])

In [10]:
lap_times_df = spark.read.schema(lap_times_schema).csv("../data/lap_times/lap_times_split*.csv")

### Step 3 - Rename and add new columns

In [13]:
lap_times_column_mapping = {
    "raceId": "race_id",
    "driverId": "driver_id"
}

lap_times_df = rename_columns(lap_times_df, lap_times_column_mapping)

In [ ]:
lap_times_df = add_ingestion_date(lap_times_df)

### Step 4 - Write to output to process container in Parquet Format

In [14]:
lap_times_df.write.mode("overwrite").parquet("../processed_data/lap_times")